In [10]:
from torch.utils.data import DataLoader

from mmengine.dataset import DefaultSampler, pseudo_collate
from mmdet.datasets.objects365 import Objects365V2Dataset
from mmpretrain.datasets.transforms import *
from mmpretrain.models import ClsDataPreprocessor

from mmengine import Config
from mmpretrain.models import build_classifier

from projects.ma_clip.datasets import InstanceDataset, LoadInstanceImage
from projects.ma_clip.models import *
from projects.clip.datasets import *
from projects.clip.models import *

In [1]:
import os
os.chdir('../')

/home/ubuntu/anaconda3/envs/mmdet_3x/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from mmpretrain.models import build_classifier
from mmpretrain.datasets import build_dataset
from mmengine import Config

In [13]:
cfg = Config.fromfile('projects/ma_clip/configs/vit-base-p16_pt-64xb64_in1k.py')
model = build_classifier(cfg.model)
model.init_weights()

05/29 16:47:43 - mmengine - INFO - load model from: data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth
05/29 16:47:43 - mmengine - INFO - Loads checkpoint by local backend from path: data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth
05/29 16:47:43 - mmengine - INFO - 
backbone.visual_projection - torch.Size([768, 512]): 
PretrainedInit: load from data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth 
 
05/29 16:47:43 - mmengine - INFO - 
backbone.positional_embedding - torch.Size([77, 512]): 
PretrainedInit: load from data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth 
 
05/29 16:47:43 - mmengine - INFO - 
backbone.text_projection - torch.Size([512, 512]): 
PretrainedInit: load from data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth 
 
05/29 16:47:43 - mmengine - INFO - 
backbone.logit_scale - torch.Size([]): 
PretrainedInit: load from data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth 
 
05/29 16:4

In [8]:
dataset = build_dataset(cfg.train_dataset)

/home/ubuntu/mnt/hxm/ovd/mmdetection/mmdet/datasets/api_wrappers/coco_api.py:24: UserWarning: mmpycocotools is deprecated. Please install official pycocotools by "pip install pycocotools"
  UserWarning)


loading annotations into memory...
Done (t=48.16s)
creating index...
index created!


In [14]:
sampler = DefaultSampler(dataset, shuffle=True)
train_loader = DataLoader(dataset=dataset, batch_size=4, sampler=sampler, collate_fn=pseudo_collate)
data_preprocessor = ClsDataPreprocessor(
    mean=[125.307, 122.961, 113.8575, 0.0],
    std=[51.5865, 50.847, 51.255, 1.0])

In [15]:
# model = model.cuda()
# data_preprocessor = data_preprocessor.cuda()
# 训练过程
for data_batch in train_loader:
#     data_batch['inputs'] = [d.cuda() for d in data_batch['inputs']]
#     data_batch['data_samples'] = [d.cuda() for d in data_batch['data_samples']]

    data_batch = data_preprocessor(data_batch, training=True)
    if isinstance(data_batch, dict):
        losses = model(**data_batch, mode='loss')
    elif isinstance(data_batch, (list, tuple)):
        losses = model(*data_batch, mode='loss')
    else:
        raise TypeError()
    print(losses)

{'loss': tensor(0.9239, grad_fn=<MulBackward0>), 'accuracy_top-1': [tensor([50.])]}
{'loss': tensor(3.7734, grad_fn=<MulBackward0>), 'accuracy_top-1': [tensor([0.])]}
{'loss': tensor(1.4454, grad_fn=<MulBackward0>), 'accuracy_top-1': [tensor([25.])]}
{'loss': tensor(1.6709, grad_fn=<MulBackward0>), 'accuracy_top-1': [tensor([50.])]}
{'loss': tensor(1.6898, grad_fn=<MulBackward0>), 'accuracy_top-1': [tensor([50.])]}
{'loss': tensor(2.5910, grad_fn=<MulBackward0>), 'accuracy_top-1': [tensor([25.])]}
{'loss': tensor(3.7946, grad_fn=<MulBackward0>), 'accuracy_top-1': [tensor([50.])]}
{'loss': tensor(2.4330, grad_fn=<MulBackward0>), 'accuracy_top-1': [tensor([25.])]}
{'loss': tensor(2.0179, grad_fn=<MulBackward0>), 'accuracy_top-1': [tensor([0.])]}
{'loss': tensor(3.2488, grad_fn=<MulBackward0>), 'accuracy_top-1': [tensor([25.])]}
{'loss': tensor(1.6471, grad_fn=<MulBackward0>), 'accuracy_top-1': [tensor([50.])]}
{'loss': tensor(2.7211, grad_fn=<MulBackward0>), 'accuracy_top-1': [tensor([25

KeyboardInterrupt: 

In [2]:
pipeline = [
    LoadInstanceImage(with_mask=True, exp_factor=1.2, channel_order='rgb'),
    ResizeEdge(scale=256, edge='short'),
    RandomCrop(crop_size=224),
    RandomFlip(prob=0.5, direction='horizontal'),
    PackInputs(algorithm_keys=['language'])
]
toy_dataset = VLDataset(
    InstanceDataset(
        Objects365V2Dataset(
            data_root='../data/Objects365/Obj365_v2/',
            data_prefix=dict(img='train/'),
            ann_file='debug/train.json'),
        filter_cfg=dict(min_size=32)),
    pipeline=pipeline)

sampler = DefaultSampler(toy_dataset, shuffle=True)
train_loader = DataLoader(dataset=toy_dataset, batch_size=2, sampler=sampler, collate_fn=pseudo_collate)
data_preprocessor = ClsDataPreprocessor(
    mean=[125.307, 122.961, 113.8575, 0.0],
    std=[51.5865, 50.847, 51.255, 1.0])

/home/ubuntu/mnt/hxm/ovd/mmdetection/mmdet/datasets/api_wrappers/coco_api.py:24: UserWarning: mmpycocotools is deprecated. Please install official pycocotools by "pip install pycocotools"
  UserWarning)


loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


In [5]:
cfg = Config.fromfile('../configs/clip/vit-base-p16_pt-64xb64_in1k.py').model
cfg.type = 'CLIPClassifier'
cfg.backbone = dict(
    type='MACLIP',
    visual_cfg=cfg.backbone,
    text_cfg=dict(
        type='TextTransformer',
        context_length=77,
        vocab_size=49408,
        width=512,
        num_heads=8,
        num_layers=12),
    output_dims=512,
    out_type='raw',
    init_cfg = dict(
        type='Pretrained', 
        checkpoint='../data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth'))
cfg.neck = dict(
    type='MACLIPNeck',
    encoder_cfg=dict(
        type='MACLIPMaskEncoder',
        embed_dims=768,
        hidden_dims=128,
    ),
    decoder_cfg=dict(
        type='MACLIPMaskDecoder',
        num_patches=196,
        embed_dims=768 * 2,
        hidden_dims=1024,
        output_dims=512,
        num_layers=8,
        num_heads=16,
        mlp_ratio=4,
    ), 
)
cfg.head= dict(
    type='CLIPClsHead',
    loss=dict(type='CLIPLoss'),
    cal_acc=True)
model = build_classifier(cfg)
model.init_weights()

05/24 17:10:03 - mmengine - INFO - load model from: ../data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth
05/24 17:10:03 - mmengine - INFO - Loads checkpoint by local backend from path: ../data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth
05/24 17:10:04 - mmengine - INFO - 
backbone.visual_projection - torch.Size([768, 512]): 
PretrainedInit: load from ../data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth 
 
05/24 17:10:04 - mmengine - INFO - 
backbone.positional_embedding - torch.Size([77, 512]): 
PretrainedInit: load from ../data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth 
 
05/24 17:10:04 - mmengine - INFO - 
backbone.text_projection - torch.Size([512, 512]): 
PretrainedInit: load from ../data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth 
 
05/24 17:10:04 - mmengine - INFO - 
backbone.logit_scale - torch.Size([]): 
PretrainedInit: load from ../data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain

In [6]:
# 训练过程
for data_batch in train_loader:
    data_batch = data_preprocessor(data_batch, training=True)
    if isinstance(data_batch, dict):
        losses = model(**data_batch, mode='loss')
    elif isinstance(data_batch, (list, tuple)):
        losses = model(*data_batch, mode='loss')
    else:
        raise TypeError()
    print(losses)

{'loss': tensor(2.7514, grad_fn=<SumBackward0>), 'accuracy_top-1': [tensor([0.])]}
{'loss': tensor(1.4961, grad_fn=<SumBackward0>), 'accuracy_top-1': [tensor([0.])]}
{'loss': tensor(2.4372, grad_fn=<SumBackward0>), 'accuracy_top-1': [tensor([50.])]}
{'loss': tensor(1.1959, grad_fn=<SumBackward0>), 'accuracy_top-1': [tensor([50.])]}
{'loss': tensor(0.9737, grad_fn=<SumBackward0>), 'accuracy_top-1': [tensor([0.])]}
{'loss': tensor(2.0520, grad_fn=<SumBackward0>), 'accuracy_top-1': [tensor([0.])]}


KeyboardInterrupt: 

In [4]:
for i in range(4 - 1, -1, -1):
    dims = 256 // 2**i
    print(dims)

32
64
128
256


In [8]:
from projects.ma_clip.clip import VLDataset

pipeline = [
    LoadInstanceImage(with_mask=True, exp_factor=1.2, channel_order='rgb'),
    ResizeEdge(scale=256, edge='short'),
    RandomCrop(crop_size=224),
    RandomFlip(prob=0.5, direction='horizontal'),
    # PackInputs(algorithm_keys=('language',))
]
toy_dataset = VLDataset(
    InstanceDataset(
        Objects365V2Dataset(
            data_root='../data/Objects365/Obj365_v2/',
            data_prefix=dict(img='train/'),
            ann_file='debug/train.json'),
        filter_cfg=dict(min_size=32)),
    pipeline=pipeline)

sampler = DefaultSampler(toy_dataset, shuffle=True)
train_loader = DataLoader(dataset=toy_dataset, batch_size=1, sampler=sampler, collate_fn=pseudo_collate)
data_preprocessor = ClsDataPreprocessor(
    mean=[125.307, 122.961, 113.8575, 0.0],
    std=[51.5865, 50.847, 51.255, 1.0])

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [9]:
toy_dataset[0]

{'img_path': '../data/Objects365/Obj365_v2/train/patch16/objects365_v2_00900003.jpg',
 'gt_label': 199,
 'bbox': [336.0317382656,
  325.89758300159997,
  657.8232421376,
  392.49353026560004],
 'text': tensor([[49406,   320,  2103,  ...,     0,     0,     0],
         [49406,   320,  1125,  ...,     0,     0,     0],
         [49406,   320,  8757,  ...,     0,     0,     0],
         ...,
         [49406,   320,  1125,  ...,     0,     0,     0],
         [49406,   320,  1125,  ...,     0,     0,     0],
         [49406,   320,  6325,  ...,     0,     0,     0]]),
 'img': array([[[ 57,  64,  48,   0],
         [ 59,  66,  48,   0],
         [ 64,  68,  51,   0],
         ...,
         [ 98,  98,  98,   0],
         [ 75,  73,  74,   0],
         [ 77,  73,  74,   0]],
 
        [[ 59,  63,  46,   0],
         [ 43,  47,  30,   0],
         [ 54,  58,  41,   0],
         ...,
         [ 97,  95,  96,   0],
         [ 75,  73,  74,   0],
         [ 81,  79,  80,   0]],
 
        [[ 59,  